In [2]:
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objects as go
import pandas as pd
import pyodbc

In [8]:
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objects as go
import pandas as pd
import pyodbc

# Conexión a la base de datos
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=localhost;'
    'DATABASE=datamart_proyecto15;'
    'UID=sa;'
    'PWD=1234567890'
)

# Función para obtener datos de las dimensiones independientes
def get_dimension_data():
    query_departments = "SELECT nombre_departamento AS Departamento FROM dimDepartamento"
    query_provinces = "SELECT nombre_provincia AS Provincia FROM dimProvincia"
    query_districts = "SELECT nombre_distrito AS Distrito FROM dimDistrito"
    query_time = "SELECT DISTINCT año AS Año, mes AS Mes FROM dimTiempo"

    df_departments = pd.read_sql(query_departments, conn)
    df_provinces = pd.read_sql(query_provinces, conn)
    df_districts = pd.read_sql(query_districts, conn)
    df_time = pd.read_sql(query_time, conn)

    return df_departments, df_provinces, df_districts, df_time

# Función para obtener datos de vulnerabilidad financiera y acceso a tecnología
def get_telecom_vulnerability_data(department=None, province=None, district=None, year=None, month=None):
    query = """
    SELECT 
        CASE 
            WHEN (be.gastoEnProductos / NULLIF(be.gananciaNeta, 0)) >= 0.5 THEN 'Alta'
            WHEN (be.gastoEnProductos / NULLIF(be.gananciaNeta, 0)) BETWEEN 0.3 AND 0.5 THEN 'Media'
            ELSE 'Baja'
        END AS RangoVulnerabilidad,
        CAST(SUM(CASE WHEN be.idInternet = 2 THEN 1 ELSE 0 END) AS FLOAT) / COUNT(*) * 100 AS PorcentajeInternet,
        CAST(SUM(CASE WHEN be.idCelular = 2 THEN 1 ELSE 0 END) AS FLOAT) / COUNT(*) * 100 AS PorcentajeCelular
    FROM factBalanceEconomico be
    JOIN dimTiempo dt ON be.idTiempo = dt.idTiempo
    WHERE be.gananciaNeta > 0
    """
    if department:
        query += f" AND be.id_departamento = (SELECT id_departamento FROM dimDepartamento WHERE nombre_departamento = '{department}')"
    if province:
        query += f" AND be.id_provincia = (SELECT id_provincia FROM dimProvincia WHERE nombre_provincia = '{province}')"
    if district:
        query += f" AND be.id_distrito = (SELECT id_distrito FROM dimDistrito WHERE nombre_distrito = '{district}')"
    if year:
        query += f" AND dt.año = {year}"
    if month:
        query += f" AND dt.mes = {month}"

    query += """
    GROUP BY 
        CASE 
            WHEN (be.gastoEnProductos / NULLIF(be.gananciaNeta, 0)) >= 0.5 THEN 'Alta'
            WHEN (be.gastoEnProductos / NULLIF(be.gananciaNeta, 0)) BETWEEN 0.3 AND 0.5 THEN 'Media'
            ELSE 'Baja'
        END
    ORDER BY RangoVulnerabilidad;
    """
    return pd.read_sql(query, conn)

# Función para obtener datos de escolarización de niños
def get_schooling_vulnerability_data(department=None, province=None, district=None, year=None, month=None):
    query = """
    SELECT 
        de.niñosEnColegio,
        AVG(be.gastoEnProductos / NULLIF(be.gananciaNeta, 0)) AS PromedioRatioGastoIngreso,
        AVG(be.gananciaNeta - be.gastoEnProductos) AS DiferenciaIngresoGasto
    FROM factBalanceEconomico be
    JOIN dimEducacionNiños de ON be.idEducacionNiños = de.idEducacionNiños
    JOIN dimTiempo dt ON be.idTiempo = dt.idTiempo
    WHERE be.gananciaNeta > 0
    """
    if department:
        query += f" AND be.id_departamento = (SELECT id_departamento FROM dimDepartamento WHERE nombre_departamento = '{department}')"
    if province:
        query += f" AND be.id_provincia = (SELECT id_provincia FROM dimProvincia WHERE nombre_provincia = '{province}')"
    if district:
        query += f" AND be.id_distrito = (SELECT id_distrito FROM dimDistrito WHERE nombre_distrito = '{district}')"
    if year:
        query += f" AND dt.año = {year}"
    if month:
        query += f" AND dt.mes = {month}"

    query += """
    GROUP BY de.niñosEnColegio
    ORDER BY de.niñosEnColegio;
    """
    return pd.read_sql(query, conn)

# Función para obtener datos de acceso a servicios
def get_wellbeing_data(department=None, province=None, district=None, year=None, month=None):
    query = """
    SELECT 
        AVG(be.gastoEnProductos / NULLIF(be.gananciaNeta, 0)) AS RatioGastoIngreso,
        CAST(SUM(CASE WHEN be.idServiciosHigienicos = 2 THEN 1 ELSE 0 END) AS FLOAT) / COUNT(*) * 100 AS ServiciosHigienicos,
        CAST(SUM(CASE WHEN be.idTelefono = 2 THEN 1 ELSE 0 END) AS FLOAT) / COUNT(*) * 100 AS Telefono,
        CAST(SUM(CASE WHEN be.idTelevision = 2 THEN 1 ELSE 0 END) AS FLOAT) / COUNT(*) * 100 AS Television
    FROM factBalanceEconomico be
    JOIN dimTiempo dt ON be.idTiempo = dt.idTiempo
    WHERE be.gananciaNeta > 0
    """
    if department:
        query += f" AND be.id_departamento = (SELECT id_departamento FROM dimDepartamento WHERE nombre_departamento = '{department}')"
    if province:
        query += f" AND be.id_provincia = (SELECT id_provincia FROM dimProvincia WHERE nombre_provincia = '{province}')"
    if district:
        query += f" AND be.id_distrito = (SELECT id_distrito FROM dimDistrito WHERE nombre_distrito = '{district}')"
    if year:
        query += f" AND dt.año = {year}"
    if month:
        query += f" AND dt.mes = {month}"

    return pd.read_sql(query, conn)

# Nueva Función para obtener datos de acceso al agua
def get_water_access_data(department=None, province=None, district=None, year=None, month=None):
    query = """
    SELECT 
        a.agua AS TipoAgua,
        COUNT(*) AS CountAgua
    FROM factBalanceEconomico be
    JOIN dimAgua a ON be.idAgua = a.idAgua
    JOIN dimTiempo dt ON be.idTiempo = dt.idTiempo
    WHERE be.gananciaNeta > 0
    """
    if department:
        query += f" AND be.id_departamento = (SELECT id_departamento FROM dimDepartamento WHERE nombre_departamento = '{department}')"
    if province:
        query += f" AND be.id_provincia = (SELECT id_provincia FROM dimProvincia WHERE nombre_provincia = '{province}')"
    if district:
        query += f" AND be.id_distrito = (SELECT id_distrito FROM dimDistrito WHERE nombre_distrito = '{district}')"
    if year:
        query += f" AND dt.año = {year}"
    if month:
        query += f" AND dt.mes = {month}"

    query += """
    GROUP BY a.agua
    ORDER BY CountAgua DESC;
    """
    return pd.read_sql(query, conn)

# Inicializar la app Dash
app = dash.Dash(__name__)
df_departments, df_provinces, df_districts, df_time = get_dimension_data()

app.layout = html.Div([
    html.H1('Dashboard de Efectos', style={'textAlign': 'center', 'color': '#34495e', 'marginBottom': '20px'}),

    # Filtros organizados en una cuadrícula 2x4
    html.Div([
        # Fila 1 de filtros
        html.Div([
            html.Div([
                html.Label('Departamento:', style={'fontWeight': 'bold'}),
                dcc.Dropdown(
                    id='department-filter',
                    options=[{'label': dept, 'value': dept} for dept in sorted(df_departments['Departamento'].unique())],
                    placeholder='Seleccione un departamento',
                    style={'backgroundColor': '#ecf0f1'}
                )
            ], style={'width': '23%', 'display': 'inline-block', 'paddingRight': '1%', 'marginBottom': '10px'}),

            html.Div([
                html.Label('Provincia:', style={'fontWeight': 'bold'}),
                dcc.Dropdown(
                    id='province-filter',
                    options=[{'label': prov, 'value': prov} for prov in sorted(df_provinces['Provincia'].unique())],
                    placeholder='Seleccione una provincia',
                    style={'backgroundColor': '#ecf0f1'}
                )
            ], style={'width': '23%', 'display': 'inline-block', 'paddingRight': '1%', 'marginBottom': '10px'}),

            html.Div([
                html.Label('Distrito:', style={'fontWeight': 'bold'}),
                dcc.Dropdown(
                    id='district-filter',
                    options=[{'label': dist, 'value': dist} for dist in sorted(df_districts['Distrito'].unique())],
                    placeholder='Seleccione un distrito',
                    style={'backgroundColor': '#ecf0f1'}
                )
            ], style={'width': '23%', 'display': 'inline-block', 'paddingRight': '1%', 'marginBottom': '10px'}),

            html.Div([
                html.Label('Año:', style={'fontWeight': 'bold'}),
                dcc.Dropdown(
                    id='year-filter',
                    options=[{'label': year, 'value': year} for year in sorted(df_time['Año'].unique())],
                    placeholder='Seleccione un año',
                    style={'backgroundColor': '#ecf0f1'}
                )
            ], style={'width': '23%', 'display': 'inline-block', 'marginBottom': '10px'})
        ], style={'width': '100%', 'display': 'flex', 'justifyContent': 'space-between'}),

        # Fila 2 de filtros
        html.Div([
            html.Div([
                html.Label('Mes:', style={'fontWeight': 'bold'}),
                dcc.Dropdown(
                    id='month-filter',
                    options=[{'label': month, 'value': month} for month in sorted(df_time['Mes'].unique())],
                    placeholder='Seleccione un mes',
                    style={'backgroundColor': '#ecf0f1'}
                )
            ], style={'width': '23%', 'display': 'inline-block', 'paddingRight': '1%', 'marginBottom': '10px'}),

            # Espacios vacíos para completar la cuadrícula 2x4
            html.Div([], style={'width': '23%', 'display': 'inline-block', 'paddingRight': '1%', 'marginBottom': '10px'}),
            html.Div([], style={'width': '23%', 'display': 'inline-block', 'paddingRight': '1%', 'marginBottom': '10px'}),
            html.Div([], style={'width': '23%', 'display': 'inline-block', 'marginBottom': '10px'})
        ], style={'width': '100%', 'display': 'flex', 'justifyContent': 'space-between'})
    ], style={'width': '90%', 'margin': 'auto', 'backgroundColor': '#bdc3c7', 'padding': '20px', 'borderRadius': '10px', 'boxShadow': '0 4px 8px rgba(0, 0, 0, 0.1)'}),

    html.Br(),

    # Gráficos organizados en una cuadrícula con dos gráficos por fila
    html.Div([
        # Primera fila de gráficos
        html.Div([
            html.Div([
                dcc.Graph(id='telecom-vulnerability-chart')
            ], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding': '1%'}),

            html.Div([
                dcc.Graph(id='schooling-vulnerability-chart')
            ], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding': '1%'})
        ], style={'width': '100%', 'display': 'flex', 'justifyContent': 'space-between'}),

        # Segunda fila de gráficos
        html.Div([
            html.Div([
                dcc.Graph(id='wellbeing-radar-chart')
            ], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding': '1%'}),

            html.Div([
                dcc.Graph(id='water-access-chart')  # Nuevo gráfico
            ], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding': '1%'})
        ], style={'width': '100%', 'display': 'flex', 'justifyContent': 'space-between'})
    ], style={'width': '90%', 'margin': 'auto'})
], style={'backgroundColor': '#f0f3f5', 'padding': '20px'})

@app.callback(
    [Output('telecom-vulnerability-chart', 'figure'),
     Output('schooling-vulnerability-chart', 'figure'),
     Output('wellbeing-radar-chart', 'figure'),
     Output('water-access-chart', 'figure')],  # Nuevo Output
    [Input('department-filter', 'value'),
     Input('province-filter', 'value'),
     Input('district-filter', 'value'),
     Input('year-filter', 'value'),
     Input('month-filter', 'value')]
)
def update_graphs(department, province, district, year, month):
    # Gráfico 1: Acceso a telecomunicaciones
    telecom_data = get_telecom_vulnerability_data(department, province, district, year, month)
    telecom_fig = go.Figure(data=[
        go.Bar(name='Acceso a Internet (%)', x=telecom_data['RangoVulnerabilidad'], y=telecom_data['PorcentajeInternet'], marker_color='blue'),
        go.Bar(name='Acceso a Celular (%)', x=telecom_data['RangoVulnerabilidad'], y=telecom_data['PorcentajeCelular'], marker_color='green')
    ])
    telecom_fig.update_layout(
        title='Vulnerabilidad financiera versus servicios de telecomunicaciones',
        barmode='group',
        plot_bgcolor='white',
        paper_bgcolor='white',
        margin=dict(l=40, r=40, t=60, b=40)
    )

    # Gráfico 2: Escolarización de niños
    schooling_data = get_schooling_vulnerability_data(department, province, district, year, month)
    # Ordenar los datos en orden descendente según PromedioRatioGastoIngreso
    schooling_data_sorted = schooling_data.sort_values(by='PromedioRatioGastoIngreso', ascending=False)
    schooling_fig = go.Figure(data=[
        go.Bar(name='Promedio Ratio Gasto/Ingreso', x=schooling_data_sorted['niñosEnColegio'], y=schooling_data_sorted['PromedioRatioGastoIngreso'], marker_color='blue'),
        go.Bar(name='Diferencia Ingreso-Gasto', x=schooling_data_sorted['niñosEnColegio'], y=schooling_data_sorted['DiferenciaIngresoGasto'], marker_color='orange')
    ])
    schooling_fig.update_layout(
        title='Vulnerabilidad financiera versus educación de niños',
        barmode='group',
        plot_bgcolor='white',
        paper_bgcolor='white',
        margin=dict(l=40, r=40, t=60, b=40)
    )

    # Gráfico 3: Bienestar general (Radar Chart)
    wellbeing_data = get_wellbeing_data(department, province, district, year, month)
    # Verificar que existen datos
    if not wellbeing_data.empty:
        radar_fig = go.Figure()
        radar_fig.add_trace(go.Scatterpolar(
            r=[
                wellbeing_data['ServiciosHigienicos'].iloc[0],
                wellbeing_data['Telefono'].iloc[0],
                wellbeing_data['Television'].iloc[0]
            ],
            theta=['Servicios Higiénicos', 'Teléfono', 'Televisión'],
            fill='toself',
            name='Bienestar'
        ))
        radar_fig.update_layout(
            title='Vulnerabilidad financiera versus Bienestar General',
            polar=dict(radialaxis=dict(visible=True, range=[0, 100])),
            showlegend=True,
            plot_bgcolor='white',
            paper_bgcolor='white',
            margin=dict(l=40, r=40, t=60, b=40)
        )
    else:
        radar_fig = go.Figure()
        radar_fig.update_layout(
            title='Vulnerabilidad financiera versus Bienestar General',
            polar=dict(radialaxis=dict(visible=True)),
            showlegend=True,
            plot_bgcolor='white',
            paper_bgcolor='white',
            margin=dict(l=40, r=40, t=60, b=40)
        )

    # Gráfico 4: Acceso al agua
    water_access_data = get_water_access_data(department, province, district, year, month)
    # Ordenar los datos en orden descendente según CountAgua
    water_access_data_sorted = water_access_data.sort_values(by='CountAgua', ascending=False)
    water_access_fig = go.Figure(data=[
        go.Bar(name='Acceso al Agua', x=water_access_data_sorted['TipoAgua'], y=water_access_data_sorted['CountAgua'], marker_color='teal')
    ])
    water_access_fig.update_layout(
        title='Acceso al Agua',
        xaxis_title='Tipo de Agua',
        yaxis_title='Cantidad',
        plot_bgcolor='white',
        paper_bgcolor='white',
        margin=dict(l=60, r=40, t=60, b=40),
        xaxis_tickangle=-45
    )

    return telecom_fig, schooling_fig, radar_fig, water_access_fig  # Retornar el nuevo gráfico

if __name__ == '__main__':
    app.run_server(debug=True)


C:\Users\claud\AppData\Local\Temp\ipykernel_3528\3980589836.py:23: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\claud\AppData\Local\Temp\ipykernel_3528\3980589836.py:24: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\claud\AppData\Local\Temp\ipykernel_3528\3980589836.py:25: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.

C:\Users\claud\AppData\Local\Temp\ipykernel_3528\3980589836.py:26: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 conn